**Sistemas Operacionais:** *Timers*

<br>

 Identificação dos autores (nome e RA):

 Daniel dos Santos Sobrinho - 813524

 Laysson Santos da Silva - 800349

 Luiz Otávio Teixeira Mello - 811967

 Murilo de Miranda Silva - 812069
<br>



# **Timer**

<u>**Timer**</u> é o dispositivo que um sistema operacional utiliza para realizar alguns serviços em um sistema computacional. Dentre eles, impedir que os programas do usuário fiquem presos em um loop infinito ou não chamem os serviços do sistema e nunca retorne o controle de volta para o SO

<br>

**A principal tarefa de um timer é interromper a CPU após um período de tempo específico**. Esse tempo é definido pelo SO e seus valores podem variar a partir de 1 ms.

<br>

# **Elementos de hardware para gerenciar o tempo**

### **Real-Time Clock (RTC)**
O Real-Time Clock (RTC), também conhecido como "Hardware Clock", é um dispositivo eletrônico, presente na maioria dos sistemas computacionais, utilizado para marcar a passagem do tempo. São não voláteis, ou seja, é capaz de manter a hora mesmo quando o dispositivo está desligado, possuem bateria interna e funcionam baseados na oscilação de um elemento cristalino. No boot, o SO sincroniza a frequência do RTC com o System Timer (Relógio do Sistema) e utiliza este para realizar as tarefas de temporização.

<br>

### **System Timer**

O System timer é um dispositivo utilizado para o **controle de eventos** que ocorrem periodicamente ou eventos que o kernel agenda para um ponto fixo no futuro. Ele dispara (geralmente chamado de hitting ou popping) em uma frequência pré-programada, chamada de **tick rate**. Quando o cronômetro do sistema dispara, ele emite uma interrupção que o kernel manipula por meio de um tratamento de interrupção especial.

<br>

Este recurso de hardware é programado para gerar interrupções a uma taxa constante. Essa taxa, dada em Hz (interrupções por segundo), é usada para incrementar uma variável interna do kernel chamada **jiffies** (será abordada posteriormente). Podemos calcular o intervalo entre interrupções causadas pelo System Timer, dividindo 1 pela taxa em Hz.

<br>

Entranto, em aplicações atuais, é comum que para o HZ (CONFIG_HZ) não  tenham valores fixos, gerenciando dinamicamente as interrupções de acordo com os timers pendente (<u>**Tickless OS**</u>). Reduzindo o overhead (despesas) e o consumo de energia causados por interrupções desnecessárias.

> ### **Tickless Kernel**
  O Tickeless Kernel é um modo de operação do Linux, em que os timers deixam de depender de interrupções periódicas do sistema para checar seus timeouts, e passa a implementar interrupções dinâmicas para tratá-los. Ou seja, em um sistema "com ticks", os timers inicializados são verificados a cada tick do sistema, enquanto que, no tickless kernel, o system timer é programado dinamicamente para gerar interrupções nos intervalos específicos dos timeouts.
>
> <br>
>
>  Por exemplo, em um sistema "com ticks", se o proxímo timer a expirar possui  timeout de 50ms, e os ticks ocorrem a  cada 1ms, então o sistema operacional deverá checar se o timeout ocorreu 50 vezes antes de executar a função do timer. Em um tickless kernel, o sistema operacional programa o system timer para disparar apenas no tempo do próximo timer, ou seja, após 50ms. Com isso, há uma única interrupção gerada para a execução do timer, no momento do timeout. Essas mudanças são eficazes, principalmente, para economia de energia do sistema, já que evita a execução desnecessária de verificações periódicas.

<br>

In [ ]:
# Configuração para manter ticks do timer constantes e periódicos,
# mesmo quando a CPU está ociosa.
config HZ_PERIODIC
	bool "Periodic timer ticks (constant rate, no dynticks)"
	help

# Configuração para um sistema "tickless" durante o estado de inatividade,
# onde as interrupções do timer só ocorrem conforme necessário,
# ajudando a economizar energia.
config NO_HZ_IDLE
	bool "Idle dynticks system (tickless idle)"
	select NO_HZ_COMMON
	help


# Configuração para um sistema "tickless" completo, onde os ticks
# do timer são desativados sempre que possível, mesmo quando a CPU
# está executando tarefas, para maximizar a eficiência energética.
# Requer configuração adicional através de parâmetros de boot.
config NO_HZ_FULL
	bool "Full dynticks system (tickless)"
	# Configuração de dependências necessárias para NO_HZ_FULL
	depends on SMP
	depends on HAVE_CONTEXT_TRACKING_USER
	depends on HAVE_VIRT_CPU_ACCOUNTING_GEN
	select NO_HZ_COMMON
	select RCU_NOCB_CPU
	select VIRT_CPU_ACCOUNTING_GEN
	select IRQ_WORK
	select CPU_ISOLATION
	help

endchoice


<br>

# **Kernel Timers**

Os Kernel Timers são sistemas de timers mantidos pelo kernel e utilizados em diversas aplicações do SO. Seu funcionamento se resume a uma função que é chamada assim que um certo "limite" de tempo é atingido, chamado de *Timeout*.
O timer é destruído após expirar, devido a isso timers são chamados muitas vezes de dynamic timers. Timers são constantemente criados e destruídos.

Para utilizá-los, devemos incluir:  ```#include <linux/timer.h>```

### ***timer_list***
Os timers do kernel Linux são definidos pela struct **timer_list**.
Ela contém em seus campos principais uma função que é passada para ser ativada no timeout, o tempo em que o timer deve expirar e ponteiros para o anterior e próximo nós.

In [ ]:
struct timer_list {

# Todos os campos que mudam durante o tempo de execução normal são agrupados no
# mesmo cacheline

	struct hlist_node		entry; # Entrada da hash list
	unsigned long				expires; # Tempo de expiração do timer
	void								(*function)(struct timer_list *); # Usa essa mesma estrutura como argumento para a função quando chamada
	u32									flags; # Flags à serem utilizados

#ifdef CONFIG_LOCKDEP
	struct lockdep_map	lockdep_map; # Estrutura para rastrear bloqueios (locks) e suas dependências, ajudando a detectar problemas como deadlocks e conflitos de concorrência entre timers
#endif
};

### **timer_base**

Responsável por manter as informações sobre os timers dessa CPU, onde cada CPU tem a sua própria estrutura. Como por exemplo, se algum está executando uma função de handling (running_timer), o próximo a ser expirado (next_expiry) também terá acesso a timer wheel.

In [ ]:
struct timer_base {
    raw_spinlock_t      lock; # Garante a sincronização no acesso
    struct timer_list   *running_timer; # Aponta para o temporizador atualmente em execução
    unsigned long       clk;
    unsigned long       next_expiry; #Mais próximo de vencer
    unsigned int        cpu;
    bool            migration_enabled; # Indica se os temporizadores podem ser migrados entre CPUs
    bool            nohz_active; # Indica se o nohz está ativo, que é um modo de otimização para que a CPU atue em baixa frequência quando não há tarefas imporantes a serem executadas
    bool            is_idle; # Indica se a CPU está ociosa
    bool            must_forward_clk; # Indica se o clk precisa ser avançado, caso o sistema tenha perdido a sincronização de tempo
    DECLARE_BITMAP(pending_map, WHEEL_SIZE); # Usado para rastrear quais wheels têm temporizadores pendentes à expirar
    struct hlist_head   vectors[WHEEL_SIZE]; # Cada entrada desse array é uma lista encadeada de temporizadores baseada no tempo de expiração (timeout) => implementação da timer wheel,        armazenada na timer base
} ____cacheline_aligned;

static DEFINE_PER_CPU(struct timer_base, timer_bases[NR_BASES]);  # DEFINE_PER_CPU (type, name) - Define uma struct timer_base por CPU
                                                                  # timer_bases[NR_BASES] - Define um array de timer_base, onde cada um deles serão usados por cada CPU; NR_BASES define a quantidade de timer_base por CPU

<br>

### **Timer Wheel**
Estrutura responsável por organizar os diversos timers do SO de forma manejável e ordenada por tempo de timeout. A timer wheel consiste de um vetor circular de listas em que cada entrada representa um intervalo de tempo. Timers são alocados nessa lista baseados em qual intervalo de tempo o seu timeout se encaixa.
A lista consiste de 8 níveis, se a frequência de interrupção for menor ou igual a 100Hz, e 9 níveis, caso contrário. Cada nível possui 64 entradas. E cada nível possui sua **granularidade** (precisão com a qual o sistema pode medir e agendar a expiração dos temporizadores em um nível específico da roda), que pode ser calculada da seguinte forma:

Granularidade = (8^(num_nvl)) * (1/Hz) ms

Em um exemplo de um x86(100Hz, 9 níveis), Note que:

1 Hz = 1 segundo. Logo:

1 / 100 = 0,01 segundos, ou seja, 10 ms. Assim:

Nível 0: 64 entradas de 10((8^0)*10) ms cada.


> 1º: 0ms - 9ms || 2º: 10 - 19ms || 3º: 20 - 29ms || ... 630ms



Nível 1: 64 entradas de 80((8^1)*10) ms cada.


> 1º: 640ms - 710ms || 2º: 720ms - 790ms || 3º: 800ms - 870ms || ... 5110ms



Nível 2: 64 entradas de 640((8^2)*10) ms cada.


> 1º: 5120ms - 5650ms || 2º: 5660ms - 6290ms || 3º: 6300ms - 6930ms || ... 40950ms



Nível 3: 64 entradas de 5120((8^3)*10) ms cada.


> 1º: 40960ms - 46070ms || 2º: 46080ms - 51190ms || 3º: 51200ms - 56310ms || ... 327670ms



Nível 4: 64 entradas de 40960((8^4)*10) ms cada.


> 1º: 327680ms - 368630ms || 2º: 368640ms - 409590ms || 3º: 409600ms - 450550ms || ... 2621430 ms



Nível 5: 64 entradas de 327680((8^5)*10) ms cada.


> 1º: 2621440ms - 2949110ms || 2º: 2949120ms - 3276790ms || 3º: 3276800ms - 3604470ms || ... 20971510 ms



Nível 6: 64 entradas de 2621440((8^6)*10) ms cada.


> 1º: 20971520ms - 23592950ms || 2º: 23592960ms - 26214390ms || 3º: 26214400ms - 28835830ms || ... 167772150 ms



Nível 7: 64 entradas de 20971520((8^7)*10) ms cada.


> 1º: 167772160ms - 188743670ms || 2º: 188743680ms - 209715190ms || 3º: 209715200ms - 230686710ms || ... 1342177270 ms

A timer-wheel utiliza um sistema em que, se o timer tem tempo de expiração maior do que o limite da wheel, ele é alocado forçadamente no último nível.

O objetivo da timer-wheel é **garantir que um temporizador não expire antes do tempo**. A expiração antecipada pode ocorrer por dois motivos:
- O temporizador foi acionado muito próximo de um "tick" (momento de verificação do temporizador)
- Truncamento do tempo de expiração ao mover o temporizador para níveis superiores da roda
Para evitar isso, arredonda-se o tempo de expiração baseado no nível (lvl).

Exemplo: Se um temporizador foi programado para expirar em 600 ms, mas ele precisa ser movido para o Nível 1, que tem uma granularidade de 80 ms, o tempo de expiração será arredondado para cima para o próximo múltiplo de 80 ms. Nesse caso, o temporizador seria ajustado para expirar em 640 ms (o próximo múltiplo de 80 ms), garantindo que ele não expire antes dos 600 ms.

In [ ]:
# Função que calcula o índice de um nível específico na timer-wheel e garante
# que o temporizador não expire mais cedo do que deveria.
static inline unsigned calc_index(unsigned long expires, unsigned lvl,
				  unsigned long *bucket_expiry)
{
	# Ajusta o tempo de expiração de acordo com o deslocamento do nível (granularidade do nível)
	expires = (expires >> LVL_SHIFT(lvl)) + 1;

	# Define o tempo exato de expiração do bucket
	*bucket_expiry = expires << LVL_SHIFT(lvl);

	# Calcula o índice do bucket somando o deslocamento de nível (LVL_OFFS) com o valor ajustado (expires & LVL_MASK)
	return LVL_OFFS(lvl) + (expires & LVL_MASK);
}

# Função que calcula em qual bucket da timer-wheel o temporizador deve ser inserido
static int calc_wheel_index(unsigned long expires, unsigned long clk,
			    unsigned long *bucket_expiry)
{
	# Calcula a diferença entre o tempo de expiração e o tempo atual
	unsigned long delta = expires - clk;
	unsigned int idx;

	# Verifica em qual nível da roda o temporizador deve ser colocado
	if (delta < LVL_START(1)) {
		# Se o delta for menor que o início do nível 1, calcula o índice no nível 0
		idx = calc_index(expires, 0, bucket_expiry);
	} else if (delta < LVL_START(2)) {
		# Se o delta for menor que o início do nível 2, calcula o índice no nível 1
		idx = calc_index(expires, 1, bucket_expiry);
	} else if (delta < LVL_START(3)) {
		# Nível 2
		idx = calc_index(expires, 2, bucket_expiry);
	} else if (delta < LVL_START(4)) {
		# Nível 3
		idx = calc_index(expires, 3, bucket_expiry);
	} else if (delta < LVL_START(5)) {
		# Nível 4
		idx = calc_index(expires, 4, bucket_expiry);
	} else if (delta < LVL_START(6)) {
		# Nível 5
		idx = calc_index(expires, 5, bucket_expiry);
	} else if (delta < LVL_START(7)) {
		# Nível 6
		idx = calc_index(expires, 6, bucket_expiry);
	} else if (LVL_DEPTH > 8 && delta < LVL_START(8)) {
		# Se houver mais de 8 níveis e o delta for menor que o início do nível 8, calcula o índice no nível 7
		idx = calc_index(expires, 7, bucket_expiry);
	} else if ((long) delta < 0) {
		# Se o delta for negativo, indica que o temporizador já expirou
		# Neste caso, o índice é o tempo atual, para que seja tratado imediatamente
		idx = clk & LVL_MASK;
		*bucket_expiry = clk;
	} else {

		# Se o delta for muito grande, garante que o temporizador seja
		# tratado dentro do limite da roda.

		if (delta >= WHEEL_TIMEOUT_CUTOFF)
			expires = clk + WHEEL_TIMEOUT_MAX;

		# Calcula o índice no último nível da roda
		idx = calc_index(expires, LVL_DEPTH - 1, bucket_expiry);
	}
	return idx;
}


<br>

# **Funções de timers do kernel**

In [ ]:
# Podemos criar um timer da seguinte forma:

struct timer_list timer;

### **timer_setup**

Após criar o timer, devemos inicilizar seus valores internos.


In [ ]:
#define timer_setup(timer, callback, flags)
	__init_timer((timer), (callback), (flags))


 # -> a função chamada ao final do macro é a do_init_timer, que efetivamente inicializa os valores do timer

static void do_init_timer(struct timer_list *timer,
			  void (*func)(struct timer_list *),
			  unsigned int flags,
			  const char *name, struct lock_class_key *key)
{
	timer->entry.pprev = NULL;
	timer->function = func;
	timer->flags = flags | raw_smp_processor_id();
	lockdep_init_map(&timer->lockdep_map, name, key, 0);
}


### **add_timer, mod_timer, mod_timer_pending, timer_reduce**


In [ ]:
# função que inicializa um timer e o coloca na timer_wheel.
# Inicia o @timer para expirar @timer->expires jiffies no futuro.
void add_timer(struct timer_list *timer)
{
	if (WARN_ON_ONCE(timer_pending(timer))) # Checagem se o @timer já está pendente. Ou seja, se ele já foi inicializado
		return;
	__mod_timer(timer, timer->expires, MOD_TIMER_NOTPENDING);
}

# modifica o tempo de expiração do timer
int mod_timer(struct timer_list *timer, unsigned long expires)
{
	return __mod_timer(timer, expires, 0);
}

# função parecida com mod_timer, mas só muda o tempo de expiração de timers pendentes
int mod_timer_pending(struct timer_list *timer, unsigned long expires)
{
	return __mod_timer(timer, expires, MOD_TIMER_PENDING_ONLY);
}

# modifica o tempo de expiração do timer, mas funciona apenas para diminuir o tempo e não aumentar
int timer_reduce(struct timer_list *timer, unsigned long expires)
{
	return __mod_timer(timer, expires, MOD_TIMER_REDUCE);
}

'''
TODAS essas funções são wrappers de __mod_timer()
'''

In [ ]:

static inline int
__mod_timer(struct timer_list *timer, unsigned long expires, unsigned int options)
{
	unsigned long clk = 0, flags, bucket_expiry;
	struct timer_base *base, *new_base;
	unsigned int idx = UINT_MAX;
	int ret = 0;

	debug_assert_init(timer);

	if (!(options & MOD_TIMER_NOTPENDING) && timer_pending(timer)) { # Identifica se o timer está pendente ou não. Caso esteja, entra na condição

		long diff = timer->expires - expires;
		# Otimização para timers que a expiração aumentou e se estabeleceu no mesmo tempo.
		# Eficiente principalmente com códigos de rede que param timers o tempo todo.
		if (!diff)
			return 1;
		if (options & MOD_TIMER_REDUCE && diff <= 0)
			return 1;

		base = lock_timer_base(timer, &flags); # bloqueia a timer_base do timer para que não haja alterações em outros timers enquanto a mudança neste aconteça

		if (!timer->function) # Identificando se o timer ainda funciona
			goto out_unlock;

		forward_timer_base(base);

		if (timer_pending(timer) && (options & MOD_TIMER_REDUCE) &&
		    time_before_eq(timer->expires, expires)) {
			ret = 1;
			goto out_unlock;
		}

		clk = base->clk;

		idx = calc_wheel_index(expires, clk, &bucket_expiry); # Re-cálculo do bucket da timer_wheel que faz parte
		if (idx == timer_get_idx(timer)) { # Verificação se o indice do timer na wheel é o mesmo de antes da alteração de tempo
			if (!(options & MOD_TIMER_REDUCE))
				timer->expires = expires;
			else if (time_after(timer->expires, expires))
				timer->expires = expires;
			ret = 1;
			goto out_unlock;
		}

	} else {
		base = lock_timer_base(timer, &flags);

		if (!timer->function) # Identificando se o timer ainda funciona
			goto out_unlock;

		forward_timer_base(base);
	}

	ret = detach_if_pending(timer, base, false);
	if (!ret && (options & MOD_TIMER_PENDING_ONLY)) # Verificação se estava pendente. Se estiver ele é desativado
		goto out_unlock;

  # Próximas linhas se referem a mudança de timer_base, caso o timer mude para outra CPU
	new_base = get_target_base(base, timer->flags);

	if (base != new_base) {

		if (likely(base->running_timer != timer)) {
			/* See the comment in lock_timer_base() */
			timer->flags |= TIMER_MIGRATING;

			raw_spin_unlock(&base->lock);
			base = new_base;
			raw_spin_lock(&base->lock);
			WRITE_ONCE(timer->flags,
				   (timer->flags & ~TIMER_BASEMASK) | base->cpu);
			forward_timer_base(base);
		}
	}

	debug_timer_activate(timer);

	timer->expires = expires;

	# Condicional para cálcular a posição na timer_wheel
	if (idx != UINT_MAX && clk == base->clk)
		enqueue_timer(base, timer, idx, bucket_expiry);
	else
		internal_add_timer(base, timer);

out_unlock:
	raw_spin_unlock_irqrestore(&base->lock, flags);

	return ret;
}

### **del_timer**

- Desativa um timer

In [ ]:
/**
 * del_timer - deactivate a timer.
 * @timer: the timer to be deactivated
 *
 * del_timer() deactivates a timer - this works on both active and inactive
 * timers.
 *
 * The function returns whether it has deactivated a pending timer or not.
 * (ie. del_timer() of an inactive timer returns 0, del_timer() of an
 * active timer returns 1.)
 */
# Função para desativar o timer
int del_timer(struct timer_list *timer)
{
	struct timer_base *base;
	unsigned long flags;
	int ret = 0;

	debug_assert_init(timer);

	if (timer_pending(timer)) { # detecta se o timer está pendente
		base = lock_timer_base(timer, &flags); # Bloqueia a timer_base
		ret = detach_if_pending(timer, base, true); # Desanexa o timer da timer_base
		raw_spin_unlock_irqrestore(&base->lock, flags); # Libera um Spinlock
	}

	return ret;
}

<br>

# **Gerenciamento de Timers**

### **Jiffies**

[Jiffies](https://elixir.bootlin.com/linux/latest/source/include/linux/jiffies.h#L80) é uma variável global que o sistema operacional utiliza para monitorar a passagem do tempo. No momento da inicialização do sistema, essa variável começa em 0 e é incrementada cada vez que ocorre uma interrupção de timer.

O tempo de funcionamento do sistema (***system uptime***) em segundos pode ser calculado dividindo o valor atual de jiffies pela taxa de interrupção do timer (jiffies/HZ).

<br>

O **tempo limite de um timer** é definido em termos de jiffies. A função associada ao timer será executada quando o valor de jiffies for igual ou superior ao tempo limite. Esse tempo limite é um valor absoluto, normalmente calculado adicionando o atraso desejado ao valor atual de jiffies.
A variável é declarada no cabeçalho: **`<linux/jiffies.h>`**.
<br>



### Modo Ticking (com ticks)
No modo **ticking**, o kernel gera interrupções periódicas em intervalos fixos, chamadas de "ticks", geralmente várias vezes por segundo. A cada tick, o sistema atualiza o contador de tempo (`jiffies`), agenda tarefas, gerencia timers e executa outras funções de manutenção. Esse método é simples e garante que as tarefas sejam verificadas e executadas regularmente, mas pode desperdiçar recursos quando o sistema está ocioso, pois os ticks continuam ocorrendo independentemente da carga.

### Modo Tickless
No modo **tickless**, o kernel desativa os ticks periódicos quando o sistema está ocioso ou com pouca carga, evitando interrupções desnecessárias. Em vez de gerar ticks constantes, o kernel agendará interrupções apenas quando houver eventos programados, como a expiração de um timer. Isso reduz o consumo de energia e melhora a eficiência, especialmente em sistemas com workloads intermitentes ou dispositivos móveis, ao permitir períodos mais longos de inatividade.

<br>

###**Timer Interrupt Handler**
É a rotina do kernel destinada a tratar as interrupções do System Timer. Realiza alguns trabalhos essenciais que devem ser feitos a cada tick do sistema, como incremento dos Jiffies, salva o wall time(tempo real), chamar outras rotinas independentes de arquitetura como, no caso do Linux, [tick_periodic()](https://elixir.bootlin.com/linux/latest/source/kernel/time/tick-common.c#L85).


In [ ]:
void tick_handle_periodic(struct clock_event_device *dev)
{
    int cpu = smp_processor_id();  # Obtém o ID da CPU atual
    ktime_t next = dev->next_event; # Armazena o tempo do próximo evento de clock configurado

    tick_periodic(cpu); # Executa o processamento do tick periódico para a CPU atual

    # Se o sistema está configurado para operar em modo "tickless" (CONFIG_TICK_ONESHOT) e o handler
    # de eventos não é tick_handle_periodic, então não há necessidade de continuar
    if (IS_ENABLED(CONFIG_TICK_ONESHOT) && dev->event_handler != tick_handle_periodic)
        return;

    # Verifica se o dispositivo de clock está no estado "one-shot"
    if (!clockevent_state_oneshot(dev))
        return;

    # Loop para programar o próximo evento de timer
    for (;;) {
        next = ktime_add_ns(next, TICK_NSEC); # Calcula o próximo evento de clock adicionando TICK_NSEC ao tempo atual

        # Programa o próximo evento de timer; se a programação for bem-sucedida, sai da função
        if (!clockevents_program_event(dev, next, false))
            return;

        # Verifica se o timekeeping (controle de tempo de alta resolução) está válido
        # Se sim, executa novamente o processamento de tick periódico
        if (timekeeping_valid_for_hres())
            tick_periodic(cpu);
    }
}


In [ ]:
static void tick_periodic(int cpu)
{
	# Verifica se a CPU atual é responsável por gerenciar o temporizador do sistema
	if (READ_ONCE(tick_do_timer_cpu) == cpu) {
		# Bloqueia o acesso ao contador de "jiffies" para evitar condições de corrida
		raw_spin_lock(&jiffies_lock);
		write_seqcount_begin(&jiffies_seq);

		# Atualiza o próximo evento de tick
		tick_next_period = ktime_add_ns(tick_next_period, TICK_NSEC);

		# Incrementa o contador de tempo do kernel
		do_timer(1);

		# Finaliza a atualização segura de "jiffies" e libera o bloqueio
		write_seqcount_end(&jiffies_seq);
		raw_spin_unlock(&jiffies_lock);

		# Atualiza o tempo do relógio do sistema
		update_wall_time();
	}

	update_process_times(user_mode(get_irq_regs()));

	# Realiza a contagem de perfil de CPU para monitoramento e análise de desempenho
	profile_tick(CPU_PROFILING);
}


In [ ]:
# Função do_timer incrementa o jiffies

void do_timer(unsigned long ticks)
{
	jiffies_64 += ticks; # Incrementa jiffies
	calc_global_load(); # updates the system’s load average statistics
}

O código abaixo trata interrupções de timer em sistemas configurados para operar em modo tickless. Ele executa tarefas periódicas somente quando necessário, economizando recursos e melhorando a eficiência energética. A função verifica o estado dos registros de interrupção, executa tarefas de manutenção do tempo, e determina se o timer deve ser reiniciado ou parado.

In [ ]:
static enum hrtimer_restart tick_nohz_handler(struct hrtimer *timer)
{
    # Obtém o ponteiro para a estrutura 'tick_sched' a partir do timer que disparou.
    struct tick_sched *ts = container_of(timer, struct tick_sched, sched_timer);

    # Recupera os registros do processador no momento da interrupção.
    struct pt_regs *regs = get_irq_regs();

    # Obtém o tempo atual em "ktime" (um tipo específico usado para manipulação de tempo no kernel).
    ktime_t now = ktime_get();

    # Executa a função para manutenção do tempo e agendamento de ticks no modo tickless.
    tick_sched_do_timer(ts, now);

    # Se os registros de interrupção estiverem disponíveis, trata o tick no modo tickless.
    # Caso contrário, define o próximo tick para 0.
    if (regs)
        tick_sched_handle(ts, regs);
    else
        ts->next_tick = 0;

    # Se a flag TS_FLAG_STOPPED estiver marcada, indica que o timer deve parar.
    if (unlikely(tick_sched_flag_test(ts, TS_FLAG_STOPPED)))
        return HRTIMER_NORESTART;  # Não reinicia o timer de alta resolução

    # Avança o timer para o próximo intervalo de tempo (TICK_NSEC).
    hrtimer_forward(timer, now, TICK_NSEC);

    # Retorna para reiniciar o timer de alta resolução.
    return HRTIMER_RESTART;
}


In [ ]:
static void tick_sched_do_timer(struct tick_sched *ts, ktime_t now)
{
	int tick_cpu, cpu = smp_processor_id(); # Obtém o ID da CPU atual

	tick_cpu = READ_ONCE(tick_do_timer_cpu); # Lê a CPU responsável pela atualização do timer

	# Verifica se a configuração "NO_HZ_COMMON" está habilitada e se nenhuma CPU é responsável pela atualização
	if (IS_ENABLED(CONFIG_NO_HZ_COMMON) && unlikely(tick_cpu == TICK_DO_TIMER_NONE)) {
#ifdef CONFIG_NO_HZ_FULL
		WARN_ON_ONCE(tick_nohz_full_running); # Gera um aviso se "tick_nohz_full_running" estiver ativo no modo "NO_HZ_FULL"
#endif
		WRITE_ONCE(tick_do_timer_cpu, cpu); # Define a CPU atual como responsável pela atualização
		tick_cpu = cpu; # Atualiza a variável com a CPU atual
	}

	# Se a CPU atual for a responsável pela atualização do timer, atualiza o jiffies (contador de tempo)
	if (tick_cpu == cpu)
		tick_do_update_jiffies64(now);

	# Verifica se o jiffies foi atualizado desde o último tick
	if (ts->last_tick_jiffies != jiffies) {
		ts->stalled_jiffies = 0; # Reinicializa o contador de jiffies estagnados
		ts->last_tick_jiffies = READ_ONCE(jiffies); # Atualiza o último valor de jiffies
	} else {
		if (++ts->stalled_jiffies == MAX_STALLED_JIFFIES) { # Incrementa o contador e verifica se atingiu o máximo
			tick_do_update_jiffies64(now); # Força uma atualização de jiffies
			ts->stalled_jiffies = 0; # Reinicializa o contador de jiffies estagnados
			ts->last_tick_jiffies = READ_ONCE(jiffies); # Atualiza o último valor de jiffies
		}
	}

	# Se o sistema está em estado de idle, marca que recebeu um tick enquanto estava ocioso
	if (tick_sched_flag_test(ts, TS_FLAG_INIDLE))
		ts->got_idle_tick = 1;
}


### **Como o kernel sabe quando um time expirou?**

In [ ]:
# update_process_times: função chamada periodicamente pelo manipulador de interrupção de timer

# Soma os tempos do processo (tempo de usuário e tempo de sistema)
# Executa as funções dos timers da CPU corrente que expiraram

# bottom-halves: mecanismo criado para que o manipulador de interrupção (interrupt handler)
# faça o mínimo de trabalho possível e retorne rapidamente


void update_process_times(int user_tick)
{
	struct task_struct *p = current; # Obtém a estrutura de dados do processo atual em execução

	# Contabiliza o tempo de CPU gasto pelo processo atual, considerando se o tempo é de usuário ou de sistema
	account_process_tick(p, user_tick);

	# Executa timers locais da CPU que atingiram o tempo de expiração (executa o "bottom-half" TIMER_SOFTIRQ)
	run_local_timers();

	# Atualiza o relógio de agendamento do RCU (Read-Copy-Update) para garantir que a contagem de tempo do RCU esteja correta
	rcu_sched_clock_irq(user_tick);

#ifdef CONFIG_IRQ_WORK
	# Se estamos dentro de um contexto de interrupção, executa trabalhos de interrupção pendentes
	if (in_irq())
		irq_work_tick();
#endif

	# Atualiza o agendador de processos, chamado periodicamente para fazer decisões de agendamento
	scheduler_tick();

	# Se o suporte a temporizadores POSIX estiver habilitado, executa os temporizadores POSIX da CPU
	if (IS_ENABLED(CONFIG_POSIX_TIMERS))
		run_posix_cpu_timers();
}

#run_local_timers()

# Verifica se há timers expirados e levanta a interrupção TIMER_SOFTIRQ se necessário,
# para ser tratada pela função run_timer_softirq()

static void run_local_timers(void)
{
	struct timer_base *base = this_cpu_ptr(&timer_bases[BASE_STD]); # Obtém a base de timers da CPU atual

	hrtimer_run_queues(); # Executa timers de HR que expiraram

	# Verifica se o próximo timer a expirar está antes de jiffies (tempo atual)
	# Levanta a interrupção TIMER_SOFTIRQ somente se necessário
	if (time_before(jiffies, base->next_expiry)) {
		if (!IS_ENABLED(CONFIG_NO_HZ_COMMON))
			return;
		# Verifica a base de timers adiáveis (deferrable) se a CPU está ativa
		base++;
		if (time_before(jiffies, base->next_expiry))
			return;
	}
	raise_softirq(TIMER_SOFTIRQ); # Levanta a interrupção TIMER_SOFTIRQ se houver timers a expirar
}

SyntaxError: invalid syntax (<ipython-input-1-c9b7f4ae998b>, line 10)

Já no modo tickless, o HRtimer sabe quando expira utilizando um loop de verificação no tempo de interrupção de alta resolução, comparando continuamente o tempo atual com os tempos de expiração definidos dos temporizadores ativos.
A função hrtimer_interrupt é chamada pelo manipulador de interrupções do dispositivo de eventos de clock configurado para o kernel, que dispara interrupções de alta resolução.

In [ ]:
void hrtimer_interrupt(struct clock_event_device *dev)
{
    # Obtém a base de temporizadores de alta resolução para a CPU atual
    struct hrtimer_cpu_base *cpu_base = this_cpu_ptr(&hrtimer_bases);
    # Variáveis para controlar o próximo evento de expiração, o tempo atual, tempo de entrada e delta
    ktime_t expires_next, now, entry_time, delta;
    unsigned long flags; # Variável para armazenar flags de interrupção
    int retries = 0; # Contador de tentativas de reprogramação

    # Verifica se o temporizador de alta resolução está ativo
    BUG_ON(!cpu_base->hres_active);
    # Incrementa o número de eventos de temporizador
    cpu_base->nr_events++;
    # Define o próximo evento de clock para o maior valor possível
    dev->next_event = KTIME_MAX;

    # Bloqueia o spinlock e salva o estado de interrupção
    raw_spin_lock_irqsave(&cpu_base->lock, flags);
    # Atualiza a base de tempo do temporizador e armazena o tempo atual e de entrada
    entry_time = now = hrtimer_update_base(cpu_base);

retry:
    cpu_base->in_hrtirq = 1; # Marca que estamos dentro do contexto de interrupção do temporizador
    cpu_base->expires_next = KTIME_MAX; # Define a próxima expiração como o valor máximo de tempo

    # Verifica se é hora de ativar o softirq do HRTIMER
    if (!ktime_before(now, cpu_base->softirq_expires_next)) {
        cpu_base->softirq_expires_next = KTIME_MAX; # Redefine o próximo evento do softirq
        cpu_base->softirq_activated = 1; # Marca o softirq como ativado
        raise_softirq_irqoff(HRTIMER_SOFTIRQ); # Levanta a interrupção softirq do temporizador
    }

    # é responsável por verificar e executar todos os high-resolution timers (hrtimers)
    # que estão programados e cuja expiração já ocorreu. Ela percorre as filas de hrtimers da CPU
    # e executa suas callbacks associadas, garantindo que as ações programadas para o momento certo sejam disparadas corretamente.
    __hrtimer_run_queues(cpu_base, now, flags, HRTIMER_ACTIVE_HARD);

    # Atualiza o próximo evento de expiração do temporizador
    expires_next = hrtimer_update_next_event(cpu_base);
    # Define a próxima expiração de eventos do temporizador
    cpu_base->expires_next = expires_next;
    # Marca que não está mais no contexto de interrupção do temporizador
    cpu_base->in_hrtirq = 0;
    # Libera o spinlock e restaura o estado de interrupção
    raw_spin_unlock_irqrestore(&cpu_base->lock, flags);

    # Programa o próximo evento de tick; se bem-sucedido, não há necessidade de novas tentativas
    if (!tick_program_event(expires_next, 0)) {
        cpu_base->hang_detected = 0; # Indica que nenhum problema de atraso foi detectado
        return; # Saída, pois o evento foi programado corretamente
    }

    # Caso o evento não seja programado corretamente, tenta novamente
    raw_spin_lock_irqsave(&cpu_base->lock, flags);
    now = hrtimer_update_base(cpu_base); # Atualiza novamente a base de tempo
    cpu_base->nr_retries++; # Incrementa o contador de tentativas de reprogramação
    if (++retries < 3) # Tenta reprogramar até 3 vezes
        goto retry; # Se falhar, volta para tentar de novo

    # Se todas as tentativas falharem, marca um hang e contabiliza o número de hangs
    cpu_base->nr_hangs++;
    cpu_base->hang_detected = 1;
    raw_spin_unlock_irqrestore(&cpu_base->lock, flags);

    # Calcula o tempo de execução da interrupção
    delta = ktime_sub(now, entry_time);
    # Atualiza o tempo máximo de atraso se o tempo atual for maior
    if ((unsigned int)delta > cpu_base->max_hang_time)
        cpu_base->max_hang_time = (unsigned int) delta;

    # Se o delta de tempo for muito grande, programa o próximo evento de tick com um limite superior
    if (delta > 100 * NSEC_PER_MSEC)
        expires_next = ktime_add_ns(now, 100 * NSEC_PER_MSEC);
    else
        expires_next = ktime_add(now, delta);

    # Reprograma o próximo evento de tick
    tick_program_event(expires_next, 1);
    # Gera um aviso indicando o tempo de execução da interrupção
    pr_warn_once("hrtimer: interrupt took %llu ns\n", ktime_to_ns(delta));
}


<br>

### **Como o kernel sabe oque executar quando os timers expiram?**

**RUN TIMER SOFTIRQ**

In [ ]:
# run_timer_softirq: Handler para a interrupção TIMER_SOFTIRQ, responsável por chamar __run_timers

static __latent_entropy void run_timer_softirq(struct softirq_action *h)
{
	struct timer_base *base = this_cpu_ptr(&timer_bases[BASE_STD]);

	__run_timers(base); # Executa timers da base padrão
	if (IS_ENABLED(CONFIG_NO_HZ_COMMON))
		__run_timers(this_cpu_ptr(&timer_bases[BASE_DEF])); # Executa timers da base adiável, se configurado
}

# __run_timers: Função que verifica e executa todos os timers expirados na CPU atual


#__run_timers - Executa todos os timers expirados (se houver) nesta CPU.
#@base: a base de timers a ser processada.

static inline void __run_timers(struct timer_base *base)
{
	struct hlist_head heads[LVL_DEPTH];
	int levels;

	# Verifica se ainda há tempo antes do próximo timer expirar
	if (time_before(jiffies, base->next_expiry))
		return;

	timer_base_lock_expiry(base); # Bloqueia o acesso à base de timers para evitar condições de corrida
	raw_spin_lock_irq(&base->lock);

	# Encontra todos os timers da CPU que precisam ser executados
	while (time_after_eq(jiffies, base->clk) &&
	       time_after_eq(jiffies, base->next_expiry)) {
		levels = collect_expired_timers(base, heads); # Coleta timers que expiraram

		# Verifica se há timers expirados para processar
		WARN_ON_ONCE(!levels && !base->next_expiry_recalc
			     && base->timers_pending);
		base->clk++;
		base->next_expiry = __next_timer_interrupt(base); # Calcula o próximo timer a expirar

		# Executa as funções associadas a cada timer expirado
		while (levels--)
			expire_timers(base, heads + levels); # Executa as funções dos timers pendentes
	}
	raw_spin_unlock_irq(&base->lock); # Libera o bloqueio da base de timers
	timer_base_unlock_expiry(base); # Desbloqueia a base de timers
}

# expire_timers: Executa as funções associadas aos timers que expiraram

static void expire_timers(struct timer_base *base, struct hlist_head *head)
{
	# Armazena o valor de clk anterior para fins de rastreamento
	unsigned long baseclk = base->clk - 1;

	# Percorre a lista de todos os timers expirados
	while (!hlist_empty(head)) {
		struct timer_list *timer;
		void (*fn)(struct timer_list *);

		timer = hlist_entry(head->first, struct timer_list, entry); # Pega o próximo timer a expirar

		base->running_timer = timer; # Marca o timer atual como em execução
		detach_timer(timer, true); # Remove o timer da lista de timers pendentes

		fn = timer->function; # Obtém a função associada ao timer

		# Verifica se a função é válida
		if (WARN_ON_ONCE(!fn)) {
			base->running_timer = NULL;
			continue;
		}

		# Executa a função do timer no contexto apropriado
		if (timer->flags & TIMER_IRQSAFE) {
			raw_spin_unlock(&base->lock);
			call_timer_fn(timer, fn, baseclk); # Chama a função do timer
			raw_spin_lock(&base->lock);
			base->running_timer = NULL; # Marca que nenhum timer está em execução
		} else {
			raw_spin_unlock_irq(&base->lock);
			call_timer_fn(timer, fn, baseclk);
			raw_spin_lock_irq(&base->lock);
			base->running_timer = NULL;
			timer_sync_wait_running(base); # Sincroniza após execução do timer
		}
	}
}


<br>

# **High-Resolution Timers**

Os chamados high-resolution timers ou hrtimers são estruturas separadas dos convencionais timers dinâmicos do kernel. OS hrtimers possuem maior precisão na marcação do expire time, já que ao invés de utilizar jiffies como base para sua contagem, eles utilizam a variável ktime_t que guarda o tempo em nanossegundos em um tamanho de 64 bits.
A [struct hrtimer](https://elixir.bootlin.com/linux/latest/source/include/linux/hrtimer.h#L118) é definida como:

In [ ]:
struct hrtimer {
	struct timerqueue_node		node; # enfileirar o hrtimer em uma fila de timers
	ktime_t				_softexpires; # tempo em nanossegundos (soft interrupt)
	enum hrtimer_restart		(*function)(struct hrtimer *); # função callback (reiniciar ou não)
	struct hrtimer_clock_base	*base; # base de tempo do hrtimer
	u8				state; # ativo, expirado, desarmado
	u8				is_rel; # timer relativo (após um intervalo) ou absoluto (tempo específico)
	u8				is_soft; # soft timer, pode ser processado fora de uma interrupção de hardware estrita
	u8				is_hard; # hard timer, processado com urgência
};

Os hrtimers não utilizam a timer_wheel como estrutura para organização dos eventos, já que a definição da timer_wheel não suporta a precisão necessária para os hrtimers(apesar de ser uma possível vantagem para uma grande parte das aplicações dos timeouts, que não necessitam de tamanha granularidade).
Como alternativa, eles utilizam uma red-black tree para realizar a ordenação dos timers por tempo de expiração.

In [ ]:
struct hrtimer_cpu_base {
	raw_spinlock_t			lock;
	unsigned int			cpu;
	unsigned int			active_bases;
	unsigned int			clock_was_set_seq;
	unsigned int			hres_active		: 1,
					in_hrtirq		: 1,
					hang_detected		: 1,
					softirq_activated       : 1,
					online			: 1;
#ifdef CONFIG_HIGH_RES_TIMERS
	unsigned int			nr_events;
	unsigned short			nr_retries;
	unsigned short			nr_hangs;
	unsigned int			max_hang_time;
#endif
#ifdef CONFIG_PREEMPT_RT
	spinlock_t			softirq_expiry_lock;
	atomic_t			timer_waiters;
#endif
	ktime_t				expires_next;
	struct hrtimer			*next_timer;
	ktime_t				softirq_expires_next;
	struct hrtimer			*softirq_next_timer;
	struct hrtimer_clock_base	clock_base[HRTIMER_MAX_CLOCK_BASES];
} ____cacheline_aligned;


A struct [hrtimer_cpu_base](https://elixir.bootlin.com/linux/v6.10.10/source/include/linux/hrtimer_defs.h#L103) são parecidos com a struct timer_base. Cada CPU terá sua própria hrtimer_cpu_base, que contém informações importantes sobre os hrtimers que estão pendentes neste processador, como por exemplo o nó cabeça da árvore rubro negra time_queue.

o mecanismo usado para temporizar os hrtimers em nanossegundos se chama clockevent. Cada CPU tem o seu próprio dispositivo de temporização de alta resolução. É nele que o kernel seta o tempo em nanossegundos para disparar a interrupção.

### **Funções para manipulação de hrtimers**


O primeiro passo para utilizar um hrtimer é inicializá-lo com a função [hrtimer_init()](https://elixir.bootlin.com/linux/latest/source/kernel/time/hrtimer.c#L1589). Nela, são definidos o timer, o modo de clock e modo do timer(one-shot ou periódico).

In [ ]:
void hrtimer_init(struct hrtimer *timer, clockid_t clock_id,
		  enum hrtimer_mode mode) # clock MONOTIC, REALTIME, BOOTTIME
{
	debug_init(timer, clock_id, mode);
	__hrtimer_init(timer, clock_id, mode);
}

Após a inicialização, o hrtimer deve ser ativado pela função [hrtimer_start()](https://elixir.bootlin.com/linux/latest/source/include/linux/hrtimer.h#L415). Essa função atribui o tempo de expiração.

In [ ]:
static inline void hrtimer_start(struct hrtimer *timer, ktime_t tim,
				 const enum hrtimer_mode mode)
{
	hrtimer_start_range_ns(timer, tim, 0, mode); # disparar com uma faixa de tolerância = 0, ou seja, disparar no valor de tim
}

Para cancelar o funcionamento de um timer, as funções [hrtimer_cancel()](https://elixir.bootlin.com/linux/latest/source/kernel/time/hrtimer.c#L1438) e [hrtimer_try_cancel()](https://elixir.bootlin.com/linux/latest/source/kernel/time/hrtimer.c#L1316)). A primeira tenta cancelar o timer e, se a função definida para o timer já tiver sido iniciada, ela espera sua conclusão. A segunda faz algo semelhante, porém se a função já tiver sido iniciada, ela retorna erro.

In [ ]:
int hrtimer_cancel(struct hrtimer *timer)
{
	int ret;

	do {
		ret = hrtimer_try_to_cancel(timer);

		if (ret < 0) # temporizador em execução
			hrtimer_cancel_wait_running(timer);
	} while (ret < 0);
	return ret; # sucesso = ret >= 0
}

In [ ]:
int hrtimer_try_to_cancel(struct hrtimer *timer)
{
    struct hrtimer_clock_base *base;
    unsigned long flags; # estado das interrupções no sistema
    int ret = -1;

    # Verifique primeiro sem bloqueio (lock). Se o temporizador não estiver ativo (nem
    # enfileirado nem executando o retorno de chamada) nada a fazer aqui.

    if (!hrtimer_active(timer)) # temporizador não está ativado
        return 0;

    base = lock_hrtimer_base(timer, &flags); # obtém o lock para garantir que não haverá condições de corrida com outras operações

    if (!hrtimer_callback_running(timer)) # Se o callback não está em execução
        ret = remove_hrtimer(timer, base, false, false); # remove da fila de temporizadores, retornando 1

    unlock_hrtimer_base(timer, &flags); # Libera o lock na base do temporizador e restaura o estado das interrupções

    return ret;

}

<br>

# **Chamadas de sistema para a suspensão de um processo por um tempo**

### **nanosleep()**

A chamada de sistema [nanosleep() ](https://man7.org/linux/man-pages/man2/nanosleep.2.html) suspende a execução da thread corrente por determinado nanosegundos.

In [ ]:
SYSCALL_DEFINE2(nanosleep, struct __kernel_timespec __user *, rqtp,
        struct __kernel_timespec __user *, rmtp) # tempo de suspensão e tempo restante, respectivamente
{
    struct timespec64 tu; # armazenar o tempo em uma representação de 64 bits

    if (get_timespec64(&tu, rqtp)) # converte o tempo especificado por rqtp do espaço do usuário para a estrutura timespec64
        return -EFAULT;

    if (!timespec64_valid(&tu)) # Verifica se a estrutura timespec64 resultante é válida
        return -EINVAL;

    current->restart_block.fn = do_no_restart_syscall; # Indica que não deve ser reiniciada se for interrompida por um sinal
    current->restart_block.nanosleep.type = rmtp ? TT_NATIVE : TT_NONE; # Verifica se há necessidade de reiniciar a syscall ou não
    current->restart_block.nanosleep.rmtp = rmtp;
    return hrtimer_nanosleep(timespec64_to_ktime(tu), HRTIMER_MODE_REL,
                 CLOCK_MONOTONIC); # Chama a função do relógio de alta precisão, com ktime, modo de timer relativo e relógio monotônico
}

A função [hrtimer_nanosleep](https://https://elixir.bootlin.com/linux/v6.10.10/source/kernel/time/hrtimer.c#L2069) suspende a execução de uma tarefa por um período de tempo preciso usando hrtimers. Ela configura o temporizador, suspende a tarefa, lida com interrupções (reiniciando a suspensão para temporizadores relativos, mas não para absolutos), e finalmente libera os recursos. Ela é usada para garantir temporização exata em sistemas que requerem alta precisão.

In [ ]:
long hrtimer_nanosleep(ktime_t rqtp, const enum hrtimer_mode mode,
                       const clockid_t clockid)
{
    struct restart_block *restart;  # Gerencia reinício de operações interrompidas
    struct hrtimer_sleeper t;       # Contém o hrtimer e dados de suspensão
    int ret = 0;
    u64 slack;

    # Obtém o slack para a tarefa atual
    slack = current->timer_slack_ns;
    if (rt_task(current))
        slack = 0;

    # Inicializa o hrtimer
    hrtimer_init_sleeper_on_stack(&t, clockid, mode);
    hrtimer_set_expires_range_ns(&t.timer, rqtp, slack);

    # O processo efetivamente é colocado para dormir
    ret = do_nanosleep(&t, mode);

    if (ret != -ERESTART_RESTARTBLOCK)
        goto out;

    if (mode == HRTIMER_MODE_ABS) {
        ret = -ERESTARTNOHAND;
        goto out;
    }

    # Prepara a reinicialização se interrompido
    restart = &current->restart_block;
    restart->nanosleep.clockid = t.timer.base->clockid;
    restart->nanosleep.expires = hrtimer_get_expires_tv64(&t.timer);
    set_restart_fn(restart, hrtimer_nanosleep_restart);

out:
    destroy_hrtimer_on_stack(&t.timer);
    return ret;
}

### **Como o valor do nanosleep() entra na Lista de Timers (hrtimer_base)**:

- O valor do tempo passado para nanosleep() é convertido internamente para ktime_t (com timespec64_to_ktime(tu)).
- Esse valor é então passado para a função hrtimer_start() dentro da implementação do hrtimer_nanosleep(), especificamente ao retornar [do_nanosleep](https://elixir.bootlin.com/linux/v6.10.10/source/kernel/time/hrtimer.c#L2021).
- Dentro do hrtimer_start(), o temporizador é configurado com o tempo de expiração, o modo de operação (relativo, no caso de HRTIMER_MODE_REL), e o relógio (como CLOCK_MONOTONIC).
- O temporizador é efetivamente adicionado à hrtimer_base pela função [enqueue_hrtimer](https:elixir.bootlin.com/linux/v6.10.10/source/kernel/time/hrtimer.c#L1081), presente dentro de [__hrtimer_start_range_ns](https://elixir.bootlin.com/linux/v6.10.10/source/kernel/time/hrtimer.c#L1214). Assim, este timer fica esperando para ser acionado e realizar sua ação pré-definida.
- O kernel usa mecanismos como o interrupt handler (manipulador de interrupções) para verificar os temporizadores que estão prontos para serem processados.
- Quando o temporizador do nanosleep() expira, o kernel acorda o processo que estava em sono e remove da lista de temporizadores.

### **usleep()**

A função [usleep()](https://codebrowser.dev/glibc/glibc/sysdeps/posix/usleep.c.html) ocasiona a dormência (bloqueio) do processo por **pelo menos** um número especificado de microsegundos. O sistema operacional somente garante que o processo irá retornar algum tempo após o parâmetro especificado. Não há garantias de quanto tempo depois de expirado o tempo definido na chamada ocorrerá o retorno da chamada.

Um processo em dormência pode também ser desbloqueado por um sinal. Se um sinal é recebido antes que o tempo especificado tenha expirado, a função usleep() retornará um valor correspondente ao tanto de tempo que resta do período de dormência.

<br>

In [ ]:
int usleep(useconds_t usec)
{
	struct timespec ts = { # conversão de microsegundos para segundos e nanossegundos
		.tv_sec = usec / 1000000,
		.tv_nsec = (usec % 1000000) * 1000,
	};

	return syscall(__NR_nanosleep, &ts, NULL); # Chama a nanosleep, pois utiliza hrtimers
}

### **alarm()**

- Chamada de sistema que envia um sinal SIGALRM para um processo em determinado tempo

- Implementação: sys_alarm() -> alarm_setitimer() -> do_setitimer() -> hrtimer_start()

- Configura um alarme baseado em tempo real que dispara um sinal SIGALRM após o tempo configurado. Se houver um alarme anterior, retorna o tempo restante desse alarme.

In [ ]:
asmlinkage unsigned long sys_alarm(unsigned int seconds) # Configura um alarme que usa o temporizador real.
{
    return alarm_setitimer(seconds);
}

unsigned int alarm_setitimer(unsigned int seconds) # Configura o temporizador e retorna o tempo restante do alarme anterior.
{
    struct itimerval it_new, it_old; # definir o alarme

#if BITS_PER_LONG < 64
    if (seconds > INT_MAX) # garantir que seconds seja 32 bits
        seconds = INT_MAX;
#endif
    it_new.it_value.tv_sec = seconds; # tempo em segundos até o alarme
    it_new.it_value.tv_usec = 0; # tempo em microssegundos até o alarme
    it_new.it_interval.tv_sec = it_new.it_interval.tv_usec = 0; # define o intervalo entre alarmes. Definido 0, único alarme

    do_setitimer(ITIMER_REAL, &it_new, &it_old); # definir o temporizador real, ou seja, quando o temporizador expira, um sinal SIGALRM é enviado ao processo.

    # Não podemos retornar 0 se tivermos um alarme pendente,
    # dando a impressão de que não havia nenhum alarme ativo ou pendente anteriormente

    if ((!it_old.it_value.tv_sec && it_old.it_value.tv_usec) ||
          it_old.it_value.tv_usec >= 500000)
        it_old.it_value.tv_sec++; # ajusta o valor do alarme antigo (it_old.it_value)
                                                            # para garantir que o retorno do tempo restante (seconds)
                                                            # seja um pouco mais alto do que o tempo restante real.
                                                            # Esse ajuste é importante para evitar que o sistema
                                                            # informe erroneamente que o alarme anterior já expirou completamente

    return it_old.it_value.tv_sec; # Retorna o tempo restante antes que o alarme atual expire.
}

int do_setitimer(int which, struct itimerval *value, struct itimerval *ovalue) # Configura o temporizador com base no tipo especificado e lida com a reinicialização
{
    struct task_struct *tsk = current; # Processo corrente
    struct hrtimer *timer; # Ponteiro para o relogio de alta precisao
    ktime_t expires; # Tempo de expiração do intervalo
    cputime_t cval, cinterval, nval, ninterval;


    # Validação dos valores de tempo

    if (!timeval_valid(&value->it_value) ||
        !timeval_valid(&value->it_interval))
        return -EINVAL;


    # ITIMER_REAL: é enviado um SIGALARM para o processo a partir do wall-clock time

    # ITIMER_VIRTUAL: envia um sinal SIGVTALRM (mede o tempo de CPU gasto pelo processo em execução.)
                                        #   para o processo depois que ele foi executado pelo tempo passado
                    #   como parâmetro (apenas o tempo efetivo que ele foi executado conta)

    # ITIMER_PROF: envia um sinal SIGPROF depois de o tempo especificado passar, ou seja, conta o tempo de CPU do processo e o tempo gasto em sistema.


    switch (which) { # Indica o tipo de temporizador
    case ITIMER_REAL:
again:
        spin_lock_irq(&tsk->sighand->siglock); # Bloqueia o acesso aos sinais do processo para garantir consistência durante a configuração.
        timer = &tsk->signal->real_timer; # Pega o temporizador real associado ao processo atual.
        if (ovalue) { # Se não for NULL, será preenchida com o valor anterior do temporizador.
            ovalue->it_value = itimer_get_remtime(timer); # Obtém o tempo restante do temporizador anterior.
            ovalue->it_interval # Intervalo do temporizador anterior.
                = ktime_to_timeval(tsk->signal->it_real_incr);
        }
        /* We are sharing ->siglock with it_real_fn() */
        if (hrtimer_try_to_cancel(timer) < 0) { # Se o temporizador não puder ser cancelado, desbloqueia e tenta novamente.
            spin_unlock_irq(&tsk->sighand->siglock);
            goto again;
        }
        expires = timeval_to_ktime(value->it_value); # Converte o valor do tempo para um formato interno (ktime_t).
        if (expires.tv64 != 0) { # Se o tempo especificado não for zero, inicia o temporizador com o novo valor.
            tsk->signal->it_real_incr = timeval_to_ktime(value->it_interval); # Define o intervalo do temporizador.
            hrtimer_start(timer, expires, HRTIMER_MODE_REL);  # inserção do temporizador na lista de eventos do kernel
                                                              # expire --> indica o tempo relativo
                                                              # HRTIMER_MODE_REL --> o temporizador começa a contar a partir do momento atual
                                                              # e expira após um período de tempo futuro
        } else
            tsk->signal->it_real_incr.tv64 = 0; # Se o tempo for zero, define o incremento do temporizador como zero.

        spin_unlock_irq(&tsk->sighand->siglock); # Desbloqueia o acesso aos sinais do processo.
        break;
    case ITIMER_VIRTUAL:
        nval = timeval_to_cputime(&value->it_value); # Converte o valor de tempo para o formato de tempo de CPU.
        ninterval = timeval_to_cputime(&value->it_interval); # Converte o intervalo do temporizador para tempo de CPU.
        read_lock(&tasklist_lock); # Bloqueia a lista de tarefas para garantir que os dados não mudem.
        spin_lock_irq(&tsk->sighand->siglock); # Bloqueia o acesso aos sinais.
        cval = tsk->signal->it_virt_expires; # Obtém o tempo de expiração do temporizador virtual anterior.
        cinterval = tsk->signal->it_virt_incr; # Obtém o intervalo do temporizador anterior.
        if (!cputime_eq(cval, cputime_zero) ||
            !cputime_eq(nval, cputime_zero)) { # Se o valor anterior ou o novo valor não for zero, ajusta o temporizador virtual.
            if (cputime_gt(nval, cputime_zero))
                nval = cputime_add(nval,
                           jiffies_to_cputime(1)); # Adiciona um pequeno incremento ao valor.
            set_process_cpu_timer(tsk, CPUCLOCK_VIRT, # Configura o temporizador virtual do processo.
                          &nval, &cval);
        }
        tsk->signal->it_virt_expires = nval; # Define o novo tempo de expiração.
        tsk->signal->it_virt_incr = ninterval; # Define o novo intervalo.
        spin_unlock_irq(&tsk->sighand->siglock); # Desbloqueia o acesso aos sinais.
        read_unlock(&tasklist_lock); # Desbloqueia a lista de tarefas.
        if (ovalue) { # Se 'ovalue' não for nulo, salva o valor anterior do temporizador.
            cputime_to_timeval(cval, &ovalue->it_value);
            cputime_to_timeval(cinterval, &ovalue->it_interval);
        }
        break;
    case ITIMER_PROF:
        nval = timeval_to_cputime(&value->it_value); # Converte o valor do temporizador para tempo de CPU.
        ninterval = timeval_to_cputime(&value->it_interval); # Converte o intervalo do temporizador.
        read_lock(&tasklist_lock); # Bloqueia a lista de tarefas.
        spin_lock_irq(&tsk->sighand->siglock); # Bloqueia o acesso aos sinais.
        cval = tsk->signal->it_prof_expires; # Obtém o tempo de expiração do temporizador profiler anterior.
        cinterval = tsk->signal->it_prof_incr; # Obtém o intervalo do temporizador profiler anterior.
        if (!cputime_eq(cval, cputime_zero) ||
            !cputime_eq(nval, cputime_zero)) { # Se o valor anterior ou o novo valor não for zero, ajusta o temporizador profiler.
            if (cputime_gt(nval, cputime_zero))
                nval = cputime_add(nval,
                           jiffies_to_cputime(1)); # Adiciona um pequeno incremento ao valor.
            set_process_cpu_timer(tsk, CPUCLOCK_PROF,
                          &nval, &cval); # Configura o temporizador profiler do processo.
        }
        tsk->signal->it_prof_expires = nval; # Define o novo tempo de expiração.
        tsk->signal->it_prof_incr = ninterval; # Define o novo intervalo.
        spin_unlock_irq(&tsk->sighand->siglock); # Desbloqueia o acesso aos sinais.
        read_unlock(&tasklist_lock); # Desbloqueia a lista de tarefas.
        if (ovalue) {  # Se 'ovalue' não for nulo, salva o valor anterior do temporizador.
            cputime_to_timeval(cval, &ovalue->it_value);
            cputime_to_timeval(cinterval, &ovalue->it_interval);
        }
        break;
    default:
        return -EINVAL;
    }
    return 0;
}

### **Como a chamada de sistema alarm() interage com as listas de temporização**:
- A chamada alarm(seconds) configura um temporizador que vai expirar após um número específico de segundos e enviará um sinal SIGALRM ao processo quando o tempo expirar.
- Internamente, essa chamada invoca a função setitimer(ITIMER_REAL, ...) para configurar um temporizador do tipo ITIMER_REAL.
- A implementação detalhada é feita pela função do_setitimer(), que interage diretamente com a timer base do kernel para inserir o temporizador na lista de temporização. Essa interação ocorre quando chama-se a função hrtimer_start(timer, expires, HRTIMER_MODE_REL) na linha 76 do código acima.
- O temporizador é inserido na timer base, uma estrutura responsável por manter as informações sobre os timers dessa CPU. O kernel usa uma timer wheel para organizar e agendar temporizadores baseados em ticks de tempo. Isso ajuda a agendar eventos futuros (neste caso, o envio de um sinal SIGALRM quando o tempo se esgota).
-  Essa lista é gerenciada internamente pelo kernel para rastrear eventos de temporização e garantir que eles sejam processados no momento correto.
- O temporizador é configurado com um modo relativo (HRTIMER_MODE_REL), o que significa que ele expira após um intervalo de tempo definido a partir do momento da sua criação.
- Quando o tempo especificado pelo usuário termina, a função de callback associada ao temporizador envia um sinal SIGALRM ao processo, interrompendo-o para notificar que um determinado intervalo de tempo terminou.

<br>

### ***Referências***


[Linux Elixir (bootlin)](https://elixir.bootlin.com/linux/latest/source) <br>

[Gerenciamento de Timers](http://www2.comp.ufscar.br/mediawiki/index.php/Gerenciamento_de_Timers) <br>

[Timers and lists in the 2.6 kernel (IBM)](https://developer.ibm.com/tutorials/l-timers-list/) <br>

[Using Kernel Timer In Linux Device Driver (EmbeTronicX)](https://embetronicx.com/tutorials/linux/device-drivers/using-kernel-timer-in-linux-device-driver/) <br>

[Linux Kernel Development](https://www.doc-developpement-durable.org/file/Projets-informatiques/cours-&-manuels-informatiques/Linux/Linux%20Kernel%20Development,%203rd%20Edition.pdf) <br>

